In [9]:
pip install torch


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torchvision
from torchvision import transforms


ModuleNotFoundError: No module named 'torch'

In [ ]:
# Load the dataset
train_dataset = torchvision.datasets.ImageFolder(root='train_dataset', transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))

test_dataset = torchvision.datasets.ImageFolder(root='test_dataset', transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]))


In [ ]:
# Define the model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# Train the model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

for epoch in range(10):
    for i, (images, targets) in enumerate(train_dataset):
        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs[0], targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
# Evaluate the model
with torch.no_grad():
    correct = 0
    total = 0

    for i, (images, targets) in enumerate(test_dataset):
        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)

        _, predicted = torch.max(outputs[0], 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    print('Accuracy:', correct / total)


In [ ]:
# Predict the fertility of new eggs
new_egg_image = Image.open('new_egg.jpg')
new_egg_image = transforms.ToTensor()(new_egg_image)
new_egg_image = new_egg_image.unsqueeze(0)

outputs = model(new_egg_image)

_, predicted = torch.max(outputs[0], 1)

if predicted == 0:
    print('The egg is fertile.')
else:
    print('The egg is infertile.')
